In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# Задаём параметры подключения к БД
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# Подключаемся к БД.
engine = create_engine(connection_string)

In [ ]:
# импортируем библиотеку обработки и анализа структурированных данных
import pandas as pd
pd.set_option('display.max_colwidth', None)

# импортируем библиотеку для визуализации данных двумерной и трёхмерной графикой
import matplotlib.pyplot as plt

# импортируем библиотеку для визуализации данных
import plotly.graph_objects as go
import plotly.express as px

# импортируем библиотеку математических вычислений
import numpy as np

# импортируем модуль для работы с датой и временем
import datetime as dt

# импортируем статистические функции
import scipy.stats as stats

# импортируем библиотеку для создания статистических графиков
import seaborn as sns

# импортируем библиотеку с математическими функциями
import math as mth

# импортируем модуль регулярных выражений
import re

# подключаем модуль для работы с JSON-форматом
import json

import requests

# импортируем карту, хороплет, и маркер
from folium import Map, Choropleth, Marker

# импортируем кластер
from folium.plugins import MarkerCluster

In [ ]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM dash_visits
        '''

In [ ]:
# Выполняем запрос и сохраняем результат
# выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок
dash_visits = pd.io.sql.read_sql(query, con = engine)

In [ ]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [ ]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


**Описание данных**

- `record_id` — первичный ключ;
- `item_topic` — тема карточки;
- `source_topic`— тема источника;
- `age_segment` — возрастной сегмент
- `dt` — дата и время
- `visits` — количество событий.

# Предобработка данных

In [ ]:
# подсчёт явных дубликатов
dash_visits.duplicated().sum()

0

In [ ]:
# количество карточек
len(dash_visits)

30745

In [ ]:
dash_visits['dt'].value_counts()

2019-09-24 18:58:00    3383
2019-09-24 18:57:00    3342
2019-09-24 18:56:00    3325
2019-09-24 18:59:00    3317
2019-09-24 18:55:00    3088
2019-09-24 19:00:00    2729
2019-09-24 18:54:00    2551
2019-09-24 18:30:00    1261
2019-09-24 18:32:00    1257
2019-09-24 18:31:00    1253
2019-09-24 18:53:00    1107
2019-09-24 18:29:00    1031
2019-09-24 18:33:00    1007
2019-09-24 18:52:00     719
2019-09-24 18:28:00     615
2019-09-24 18:34:00     576
2019-09-24 18:35:00     184
Name: dt, dtype: int64

# Исследуем данные

In [ ]:
item_topic = dash_visits.groupby(['item_topic', 'source_topic'])['record_id'].count().reset_index()

In [ ]:
# строим гистограмму
fig = px.bar(item_topic, # загружаем данные
             x='item_topic', # указываем столбец с данными для оси X
             y='record_id', # указываем столбец с данными для оси Y
             color='source_topic', # обозначаем категорию для разделения цветом
             title='Темы карточек', # указываем заголовок  
             labels=dict(record_id="Кол-во тем карточек", item_topic="Категории тем карточек"), # подпись для оси X и Y
             width=1200, # размер по ширине
             height=600 # размер по высоте
             )
fig.update_xaxes( tickangle=90) # задаём угол наклона текста
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show() # выводим график

In [ ]:
# посмотрим как распределены темы карточек
# выведем Топ-3
dash_visits.groupby('item_topic')['visits'].sum().sort_values(ascending=False).head()

item_topic
Наука               21736
Отношения           20666
Интересные факты    19942
Общество            19640
Подборки            17772
Name: visits, dtype: int64

In [ ]:
# количество карточек с темой наука
len(dash_visits.query('item_topic == "Наука"'))

1505

In [ ]:
source_topic = dash_visits.groupby(['source_topic', 'item_topic'])['record_id'].count().reset_index()

In [ ]:
# строим гистограмму
fig = px.bar(item_topic, # загружаем данные
             x='source_topic', # указываем столбец с данными для оси X
             y='record_id', # указываем столбец с данными для оси Y
             color='item_topic', # обозначаем категорию для разделения цветом
             title='Темы источников', # указываем заголовок  
             labels=dict(record_id="Кол-во тем источников", item_topic="Категории тем источников"), # подпись для оси X и Y
             width=1200, # размер по ширине
             height=600 # размер по высоте
             )
fig.update_xaxes( tickangle=90) # задаём угол наклона текста
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show() # выводим график

In [ ]:
# посмотрим как распределены темы источника
# выведем Топ-3
dash_visits['source_topic'].value_counts().head(3)

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Name: source_topic, dtype: int64

In [ ]:
dash_visits.groupby('source_topic')['visits'].sum().sort_values(ascending=False).head()

source_topic
Семейные отношения    33309
Россия                29831
Полезные советы       27412
Путешествия           24124
Знаменитости          23945
Name: visits, dtype: int64

# Выгрузим данные из таблицы `dash_visits` в файл `dash_visits.csv`

In [ ]:
from IPython.display import HTML
import base64   

def create_download_link(dash_visits, title = "Download CSV file", filename = "dash_visits.csv"):  
    csv = dash_visits.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(dash_visits)

Output hidden; open in https://colab.research.google.com to view.